In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import scipy.linalg
from uncertainties import ufloat, correlated_values
from uncertainties import unumpy as unp
from glob import glob
from tqdm import tqdm
from copy import copy
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pickle
import uncertainties
import statsmodels.stats.correlation_tools
from uncertainties import covariance_matrix

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#from analysis_vcb.selection import sig_id_to_label
from academy.analysis.plotting import init_plot_style, add_watermark, channel_label
from academy.analysis.settings import *
from academy.analysis.constants import *

init_plot_style()

In [ ]:
df_mc = pd.read_hdf('AcademySample.h5')

In [ ]:
plotoutdir = "output/Unfolding"
try:
    os.makedirs(plotoutdir)
except FileExistsError:
    pass

In [ ]:
channels = [
    15,  #S B0 -> D*+ e nu
    16,  #S B0 -> D*+ mu nu
    #17,  #+ B+ -> D*0 e nu
    #18,  #+ B+ -> D*0 mu nu
]

channels_Dstar = [
    #21,  #+ D*0 -> D0 pi0
    23,  #S D*+ -> D0 pi+
    24,  #S D*+ -> D+ pi0
]  
channels_D = [
    31,  #S D+ -> K- pi+ pi+
    32,  #N D+ -> K- pi+ pi+ pi0
    33,  #N D+ -> K- pi+ pi+ pi+ pi-
    34,  #N D+ -> Ks pi+
    35,  #N D+ -> Ks pi+ pi0
    36,  #N D+ -> Ks pi+ pi+ pi-
    37,  #N D+ -> Ks K^+
    38,  #N D+ -> K+ K- pi^+
    41,  #S D0 -> K- pi+
    42,  #S D0 -> K- pi+ pi0
    43,  #S D0 -> K- pi+ pi+ pi-
    44,  #N D0 -> K- pi+ pi+ pi- pi0
    45,  #N D0 -> Ks pi0
    46,  #N D0 -> Ks pi+ pi-
    47,  #N D0 -> Ks pi+ pi- pi0
    48,  #N D0 -> K+ K+
]

# Setup

In [ ]:
fit_variable = "m2RecoilSignalSide_after_smearing" 
fit_variable_label = r"$M_\mathrm{miss}^2$"
fit_variable_unit = r"GeV$^2$/$c^4$"
fit_bins = np.array([-1.0, -0.25, 0.25, 0.75, 1.25, 2.0])
fit_range = (min(fit_bins), max(fit_bins))
fit_vois = ["wReco", "costhetalReco"]

In [ ]:
df_mc = df_mc.query(f"{fit_range[0]} < {fit_variable} < {fit_range[1]}")

In [ ]:
markers = {
    15: "v",
    16: "s",   
}

In [ ]:
data_type="data"

In [ ]:
plotoutdir = os.path.join(plotoutdir, data_type)
try:
    os.makedirs(plotoutdir)
except FileExistsError:
    pass
print(plotoutdir)

In [ ]:
yields = pd.read_pickle(os.path.join(f"output/Fitting/data", "yields.pkl"))

In [ ]:
correlations = {}
for channel in channels:
    correlations[channel] = statsmodels.stats.correlation_tools.corr_nearest(  # Fix negative eigenvalues
        pd.read_pickle(os.path.join(f"output/Fitting/{data_type}", f"correlation_{channel}.pkl")).values,
        threshold=2e-7
    )

In [ ]:
covariances = {}
for channel in channels:
    covariances[channel] = np.outer(
        unp.nominal_values(yields.loc[channel_label[channel], '$\sigma_\mathrm{tot}$']), 
        unp.nominal_values(yields.loc[channel_label[channel], '$\sigma_\mathrm{tot}$'])
    ) * correlations[channel]

In [ ]:
results = {}
for channel in channels:
    results[channel] = correlated_values(
        unp.nominal_values(yields.loc[channel_label[channel], '$\nu_\mathrm{sig}$']),
        covariances[channel]
    )

# Theory

In [ ]:
from academy.formfactors.BGL import BToDStarBGL
from academy.rates.BtoV import BtoV

In [ ]:
theory_bin_boundaries = {}
theory_expectation = {}
full_theory_bgl = {}

In [ ]:
param_central_values = np.array(
        [
            1.00e-3,  # a0
            -2.35e-3,  # a1
            0.511e-3,  # b0
            0.67e-03,  # b1
            3.0e-04,  # c1
            -3.68e-03,  # c2
        ]
    )  # type: np.array

In [ ]:
m_Bzero = 5.27963
m_Dstarplus = 2.01026

vcb_cln = 37.4e-3
vcb_bgl = 41.6558e-3

In [ ]:
m_Bzero = 5.27963
m_Dstarplus = 2.01026

vcb_cln = 37.4e-3
vcb_bgl = 41.6558e-3

In [ ]:
theory_bin_boundaries = {}
theory_expectation = {}
full_theory_bgl = {}

In [ ]:
btodst_bgl_ff = BToDStarBGL(
    m_B=m_Bzero,
    m_V=m_Dstarplus,
    exp_coeff_a = param_central_values[0:2],
    exp_coeff_b = param_central_values[2:4],
    exp_coeff_c = param_central_values[4:6],
)

btodst_bgl = BtoV(btodst_bgl_ff, Vcb=vcb_cln)

In [ ]:
theory_bin_boundaries["mixed"] = {}
for voi in fit_vois:
    bin_edges = copy(bin_edges_all[voi])
    if voi == "wReco":
        bin_edges[0]  = btodst_bgl_ff.kinematics.w_range_numerical_stable[0]
        bin_edges[-1] = btodst_bgl_ff.kinematics.w_range_numerical_stable[1]
        
    integration_boundaries = []
    for bin_boundaries in zip(bin_edges, bin_edges[1:]):
        integration_boundaries.append(bin_boundaries)
    theory_bin_boundaries["mixed"][voi] = integration_boundaries

In [ ]:
# Reweight total rate, because Vcb is in the rate calculation but we want the BR in the MC
total_rate_theory_bgl_weight = (BR_B0_to_Dstplus_lepton.nominal_value / tauBzero) / btodst_bgl.Gamma()

# Acceptance

## Exercise 3: Acceptance Correction

3 a) Use the defined `btodst_bgl` effort class to calculate the expected rate in each bin given the underlying theory model in our MC (BGL, with the defined coefficients above). Nota bene: Divide the expectation by the bin width. Use the defined functions `DGamma_Dw` and `DGamma_DcosL`.

In [ ]:
theory_expectation["mixed"] = {}
theory_expectation["mixed"]["BGL"] = {}
theory_expectation["mixed"]["BGL"]["wReco"] = None # Ex. 3a)
theory_expectation["mixed"]["BGL"]["costhetalReco"] = None # Ex. 3a) 

In [ ]:
for channel in channels:
    full_theory_bgl[channel] = {}
    full_theory_bgl[channel] = np.array([theory_expectation["mixed"]["BGL"][voi] for voi in fit_vois]).flatten() * total_rate_theory_bgl_weight

In [ ]:
full_theory_bgl

In [ ]:
linestyles = {
    15: "-",
    16: "--",
}

acceptance_corrections = {}
acceptance_corrections_integrated = {}

In [ ]:
bin_widths = {}
foo = []
for voi in fit_vois:
    foo.append([x[1] - x[0] for x in theory_bin_boundaries["mixed"][voi]])
bin_widths["mixed"] = np.array(foo).flatten()
del foo

3 b) Calculate the number of signal events, that survive selection and reconstruction. Use the Signal MC for this purpose. 

The number of expected events in bin $i$ is $n_i = \sum_j^\mathrm{events} w_j$, and the uncertainty is $\sigma_i(n_i) = \sqrt{\sum_j^\mathrm{events} w_j^2}$.

The acceptance is then given by the reconstructed number of events over expected number of events (use the defined value in the cell below. Why do we multiply the factors to the rate we calculated in 3a ?)

3 c) Why do we multiply the expected rate with 2 * N_BB * BR(B0 --> D* l nu) * tauBzero * bin_width? Explain each factor

In [ ]:
fig, ax = plt.subplots(dpi=130, figsize=(6.4, 4.4))

for channel in channels:

    histogram = []
    histogram_unc = []
    for voi in fit_vois:
        bin_edges = copy(bin_edges_all[voi])
        query = f"SIG_ID in (4.1, 4.2) and daughter__bo1__cm__spextraInfo__bodecayModeID__bc__bc == {channel} and {fit_range[0]} < {fit_variable} < {fit_range[1]}"
        
        # 3 a) Histogram the signal MC (you can query with df_mc.query(query) to select the signal MC. Use np.histogram, remember that we have pre-defined the bin via `bin_edges`, and that the weights are given in the column `__weight_overall__`.
    
    # Flatten the histogram(_unc) list we defined above.
    signal_mc_histogram = np.array(histogram).flatten()
    signal_mc_histogram_unc = np.array(histogram_unc).flatten()

    
    # 3 c) Why do we multiply here with 2 * N_BB * BR(B0 --> D* l nu) * tauBzero * bin_width? Explain each factor
    expected = unp.nominal_values(full_theory_bgl[channel] * 2 * N_BB.nominal_value * BR_GENERIC_MIXED.nominal_value * tauBzero * bin_widths["mixed"])
    
    acceptance_corrections[channel] = unp.uarray(
            signal_mc_histogram,
            signal_mc_histogram_unc ** 0.5
    ) / expected[:20]
    acceptance = unp.nominal_values(acceptance_corrections[channel])
    acceptance_unc = unp.std_devs(acceptance_corrections[channel])
    ax.step(range(-1,21), 1e6*np.array([acceptance[0], *acceptance, acceptance[-1]]), ls=linestyles[channel], where="mid", label=channel_label[channel], lw=1)
    ax.bar(
        range(0, 20),
        height=1e6 * 2 * acceptance_unc,
        bottom=1e6 * (acceptance - acceptance_unc),
        width=1, alpha=0.3,
    )

tmp = {}
for voi in fit_vois:
    tmp[voi] = np.array([(lower, upper) for lower, upper in zip(bin_edges_all[voi], bin_edges_all[voi][1:])])
tmp = np.array([b for voi in fit_vois for b in tmp[voi]])

# We want to show all ticks...
ax.set_xticks(np.arange(len(tmp)))
# ... and label them with the respective list entries
ax.set_xticklabels([f"[{lower:.2f}, {upper:.2f}]" for lower, upper in tmp], rotation=90, fontsize=6)
ax.xaxis.set_minor_locator(MultipleLocator(1))

ax.set_xlim(-0.5, 19.5)
ax.set_ylim(0, 200)
ax.axvline(9.5, color="black")

ax.text(0.8, 0.85, "Normal\nApproximation\nIntervals", fontsize=8, transform=ax.transAxes)
ax.text(0.00+0.12, 1.02, r"$w$", fontsize=10, transform=ax.transAxes)
ax.text(0.55+0.08, 1.02, r"$\cos \theta_\ell$", fontsize=10, transform=ax.transAxes)

ax.set_ylabel(r"Acceptance $\times 10^6$")
ax.legend(frameon=False, fontsize="x-small", ncol=1, loc='upper left')

plt.tight_layout()
plt.savefig(f"{plotoutdir}/acceptance.pdf", transparent=True)
plt.savefig(f"{plotoutdir}/acceptance.png", transparent=True)   
plt.show()
plt.close()

In [ ]:
def make_plot_pretty(ax, bin_edges_all):
    
    tmp = {}
    for voi in fit_vois:
        tmp[voi] = np.array([(lower, upper) for lower, upper in zip(bin_edges_all[voi], bin_edges_all[voi][1:])])
    tmp = np.array([b for voi in fit_vois for b in tmp[voi]])
    
    # We want to show all ticks...
    ax.set_xticks(np.arange(len(tmp)))
    # ... and label them with the respective list entries
    ax.set_xticklabels([f"[{lower:.2f}, {upper:.2f}]" for lower, upper in tmp], rotation=90, fontsize=6)
    ax.xaxis.set_minor_locator(MultipleLocator(1))

    ax.set_xlim(-0.5, 19.5)

    ax.axvline(9.5, color="black")

    ax.text(0.00+0.12, 1.02, r"$w$", fontsize=10, transform=ax.transAxes)
    ax.text(0.55+0.08, 1.02, r"$\cos \theta_\ell$", fontsize=10, transform=ax.transAxes)
    
    ax.legend(frameon=False, fontsize="x-small", ncol=1, loc='upper left')
    return ax

# Unfolding

In [ ]:
from academy.analysis.unfolding import extract_bs_spectrum

## Exercise 4: Migration Matrix

Next we want to unfold our background subtracted spectrum. For this we want to calculate the migration matrix. Use the pre-defined function below and fill out the function body.
The matrix should have the form `w Reco` on the "x-axis", and `w MC` on the "y-axis". (This can be done differently, but the follow-up code assumes this.

In [ ]:
def determine_migration_martrix(df, bin_edges, col_mc, col_reco, weight_column="__weight_overall__"):

    return None # Ex 4 Return Migration Matrix


In [ ]:
migration_matrices = {}

for channel in channels:
    migration_matrices[channel] = {}
    for voi in fit_vois:
        query = f"SIG_ID in (4.1, 4.2) and {fit_range[0]} < {fit_variable} < {fit_range[1]} and daughter__bo1__cm__spextraInfo__bodecayModeID__bc__bc == {channel}"
        migration_matrices[channel][voi] = determine_migration_martrix(df_mc.query(query), bin_edges_all[voi], voi_mc[voi], voi)

In [ ]:
full_migrations = {}

for channel in channels:
    n_bins = len(fit_vois)*10
    full_migration = np.zeros((n_bins, n_bins))
    for i, voi in enumerate(fit_vois):
        full_migration[i*10:(i+1)*10, i*10:(i+1)*10] = migration_matrices[channel][voi]
    full_migrations[channel] = full_migration

In [ ]:
mc_distributions = {}

for channel in channels:
    mc_distributions[channel] = {}
    for voi in fit_vois:
        query = f"SIG_ID in (4.1, 4.2) and {fit_range[0]} < {fit_variable} < {fit_range[1]} and daughter__bo1__cm__spextraInfo__bodecayModeID__bc__bc == {channel}"
        mc_distributions[channel][voi] = np.histogram(df_mc.query(query)[voi_mc[voi]], bins=bin_edges_all[voi], weights=df_mc.query(query)["__weight_overall__"])[0]
        
        
mc_reco_distributions = {}

for channel in channels:
    mc_reco_distributions[channel] = {}
    for voi in fit_vois:
        query = f"SIG_ID in (4.1, 4.2) and {fit_range[0]} < {fit_variable} < {fit_range[1]} and daughter__bo1__cm__spextraInfo__bodecayModeID__bc__bc == {channel}"
        mc_reco_distributions[channel][voi] = np.histogram(df_mc.query(query)[voi], bins=bin_edges_all[voi], weights=df_mc.query(query)["__weight_overall__"])[0]

In [ ]:
full_mc_distributions = {}

for channel in channels:
    full_mc_distributions[channel] = np.concatenate(tuple(mc_distributions[channel][voi] for voi in fit_vois))
    
full_mc_reco_distributions = {}

for channel in channels:
    full_mc_reco_distributions[channel] = np.concatenate(tuple(mc_reco_distributions[channel][voi] for voi in fit_vois))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

for channel in channels:
    for voi in fit_vois:
        migration_matrix = migration_matrices[channel][voi]
        fig, ax = plt.subplots(dpi=130, figsize=(6.4, 4.4 / 0.8))
        im = ax.imshow(migration_matrix, vmin=0, vmax=1, cmap=plt.get_cmap("Reds"))

        # We want to show all ticks...
        bin_edges = bin_edges_all[voi]
        ax.set_xticks(np.arange(len(bin_edges)) - 0.5)
        ax.set_yticks(np.arange(len(bin_edges)) - 0.5)
        # ... and label them with the respective list entries
        ax.set_xticklabels([f"{x:.2f}" for x in bin_edges])
        ax.set_yticklabels([f"{x:.2f}" for x in bin_edges])

        ax.set_xlabel(f"{voi_labels[voi]} Reco")
        ax.set_ylabel(f"{voi_labels[voi]} MC")

        # Create colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="3%", pad=0.05)
        cbar = ax.figure.colorbar(im, ax=ax, cax=cax)
        #cbar.ax.set_ylabel("...", rotation=-90, va="center")

        # Turn spines off and create white grid.
        for edge, spine in ax.spines.items():
            spine.set_visible(False)

        ax.set_xticks(np.arange(migration_matrix.shape[1]+1)-.5, minor=True)
        ax.set_yticks(np.arange(migration_matrix.shape[0]+1)-.5, minor=True)
        ax.grid(which="major", color="black", linestyle='-', linewidth=1.2)
        ax.tick_params(which="minor", bottom=False, left=False)

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")

        # Loop over data dimensions and create text annotations.
        for i in range(len(bin_edges)-1):
            for j in range(len(bin_edges)-1):
                text = ax.text(j, i, f"{migration_matrix[i, j]:.3f}",
                               ha="center", va="center", color="black", fontsize=8)


        add_watermark(ax, channel_label[channel])
        plt.tight_layout()
        plt.savefig(f"{plotoutdir}/migration_matrix_channel{channel}_{voi}.pdf", transparent=True)
        plt.savefig(f"{plotoutdir}/migration_matrix_channel{channel}_{voi}.png", transparent=True) 
        plt.show()
        plt.close()

# Exercise 5: Apply Unfolding and Acceptance Correction

Below is the background subtracted spectrum from the previous session.

In [ ]:
fig, ax = plt.subplots(dpi=130, figsize=(6.4, 4.4))

for i, channel in enumerate([15, 16]):
    
    result = results[channel]
    
    ax.errorbar(
        x=[x +(0.2*i - 0.2) for x in range(0, 20)],
        y=unp.nominal_values(result),
        yerr=unp.std_devs(result),
        marker=markers[channel], ls="", markeredgecolor='black', label=channel_label[channel],
        
    )
    
    histogram = full_mc_reco_distributions[channel]
    ax.step(range(-1,21), np.array([histogram[0], *histogram, histogram[-1]]), 
            ls="solid", lw=1, where="mid", color=plt.gca().lines[-1].get_color())

make_plot_pretty(ax, bin_edges_all)
ax.set_ylim(0, 500)
ax.set_ylabel("Background Subtracted Events")


plt.tight_layout()
plt.savefig(f"{plotoutdir}/bs_spectrum.pdf", transparent=True)
plt.savefig(f"{plotoutdir}/bs_spectrum.png", transparent=True)   
plt.show()
plt.close()

Ex. 5a) Apply the unfolding to the background subtracted yields

In [ ]:
fig, ax = plt.subplots(dpi=130, figsize=(6.4, 4.4))

for i, channel in enumerate([15, 16]):
    
    # define inv_m (our unfolding matrix)
    # inv_m = ...
    
    # unfold our background subtracted spectrum. The results are available in results[channel] for each channel
    # result = ...
    
    ax.errorbar(
        x=[x +(0.2*i - 0.2) for x in range(0, 20)],
        y=unp.nominal_values(result),
        yerr=unp.std_devs(result),
        marker=markers[channel], ls="", markeredgecolor='black', label=channel_label[channel],
        
    )
    
    histogram = full_mc_distributions[channel]
    ax.step(range(-1,21), np.array([histogram[0], *histogram, histogram[-1]]), 
            ls="solid", lw=1, where="mid", color=plt.gca().lines[-1].get_color())
        

make_plot_pretty(ax, bin_edges_all)
ax.set_ylim(0, 500)
ax.set_ylabel(r"Unfolded ($\mathcal{M}$) Events")

plt.tight_layout()
plt.savefig(f"{plotoutdir}/unfolded_m_spectrum.pdf", transparent=True)
plt.savefig(f"{plotoutdir}/unfolded_m_spectrum.png", transparent=True)
plt.show()
plt.close()

In [ ]:
def plot_theory(ax, full_theory, bin_widths):
    theory = full_theory[15] * bin_widths["mixed"]
    ax.step(range(-1,21), 1e15*np.array([theory[0], *theory[:20], theory[-1]]), 
            lw=1, ls="-", where="mid", color="black", label="$B^0$ BGL Theory")
    return ax

In [ ]:
def get_factor(channel):
    factor = (2*N_BB.nominal_value * BR_GENERIC_MIXED.nominal_value) * tauBzero
    return factor * np.ones(20)

Ex. 5b) Apply the unfolding *and* acceptance correction.

In [ ]:
fig, ax = plt.subplots(dpi=130, figsize=(6.4, 4.4))

for i, channel in enumerate(channels):
    
    # define inv_m (our unfolding matrix)
    # inv_m = ...
    
    # unfold our background subtracted spectrum. The results are available in results[channel] for each channel
    # result = ...
    
    factor = get_factor(channel) # This number translates yields to rates. The you have to divide by that number.
    acceptance = acceptance_corrections[channel] # This is our determined acceptance function
    
    # divide the result by both the factor and the acceptance correction
    # result = ...
    
    ax.errorbar(
        x=[x +(0.2*i - 0.2) for x in range(0, 20)],
        y=1e15*unp.nominal_values(result),
        yerr=1e15*unp.std_devs(result),
        marker=markers[channel], ls="", markeredgecolor='black', label=channel_label[channel],
    )

plot_theory(ax, full_theory_bgl, bin_widths)
make_plot_pretty(ax, bin_edges_all)
ax.set_ylim(0, 5)
ax.set_ylabel(r"($\mathcal{M}$) $\mathrm{d} \Gamma / \mathrm{d}x$ $\times 10^{-15}$ GeV$^{-1}$")

plt.tight_layout()
plt.savefig(f"{plotoutdir}/acceptance_corrected_m_spectrum.pdf", transparent=True)
plt.savefig(f"{plotoutdir}/acceptance_corrected_m_spectrum.png", transparent=True)   
plt.show()
plt.close()

# Everything at once

Here we basically re-do everything we did above, with the exception that we do it globally instead of channel by channel. This is useful to properly carry the systematic affects in the unfolding process. Conceptually this is the same thing.

In [ ]:
full_yields = yields['$\nu_\mathrm{sig}$'].values

In [ ]:
full_covariance = np.outer(
    yields['$\sigma_\mathrm{tot}$'].values, yields['$\sigma_\mathrm{tot}$'].values
) * pd.read_pickle(os.path.join(f"output/Fitting/{data_type}", f"correlation_full.pkl"))

In [ ]:
fit_yields = np.array(correlated_values(full_yields, full_covariance))

In [ ]:
inv_m = scipy.linalg.block_diag(*[np.linalg.inv(full_migrations[channel]) for channel in channels])
acceptance_correction = np.array([acceptance_corrections[channel] for channel in channels]).flatten()
factor = np.array([get_factor(channel) for channel in channels]).flatten()
bw = bin_widths["mixed"]

unfolded = inv_m @ fit_yields
corrected =  np.array([r / a / f for r, a, f in zip(unfolded, acceptance_correction, factor)])

In [ ]:
np.save(os.path.join(plotoutdir, f"minv_central_values_full.npy"), unp.nominal_values(corrected))

In [ ]:
np.save(os.path.join(plotoutdir, f"minv_covariance_full.npy"), uncertainties.covariance_matrix(corrected))

In [ ]:
corrected